In [1]:
%cd ../..
%run cryptolytic/notebooks/init.ipynb
import pandas as pd
import tensorflow as tf
import cryptolytic.util as util
import cryptolytic.start as start
import cryptolytic.viz.plot as plot
import cryptolytic.data.sql as sql
import cryptolytic.data as d
from cryptolytic.util import *
import cryptolytic.data.historical as h
import cryptolytic.model as m
import cryptolytic.model.lstm_framework as lstm
import cryptolytic.model.model_framework as mfw
import cryptolytic.model.data_work as dw
import cryptolytic.model.hyperparameter as hyper
import cryptolytic.model.cron_model as cron_model

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline


import cryptolytic.model.xgboost_arb_model as xam
import cryptolytic.data.aws as aws
import pickle

/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
mfw

<module 'cryptolytic.model.model_framework' from '/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds/cryptolytic/model/model_framework.py'>

In [5]:
params = {
        'history_size': 400,
        'lahead': 12*3,
        'step': 1,
        'period': 300, # 5 minute candles
        'batch_size': 200,
        'train_size': 10000,
        'ncandles': 5000
}
start = '01-12-2019'

for exchange_id, trading_pair in h.yield_unique_pair(return_api=False):
    model_path = mfw.get_model_path('models', 'trade', exchange_id, trading_pair, '.pkl')
    model = pickle.load(open(model_path, 'rb'))
    
    df, dataset = h.get_latest_data(
                exchange_id, 
                trading_pair, 
                params['period'], 
                n=n)
    
    df, dataset = h.get_data(exchange_id, trading_pair, period, start, n=params['train_size'])    
    
    if df is None:
        break

    target = df.columns.get_loc('close')

    # Find the x and y train and test data
    x_train, y_train, x_test, y_test = xgmod.data_splice(df)
    model = xgmod.fit_model(model, x_train, y_train)
    y_preds = model.predict(x_test)
    score = accuracy_score(y_test, y_preds)
    
    print(y_preds, score)

AttributeError: module 'cryptolytic.model.model_framework' has no attribute 'get_model_path'

In [ ]:
# labels for confusion matrix
unique_y_test = y_test.unique().tolist()
unique_y_preds = list(set(y_preds))
labels = list(set(unique_y_test + unique_y_preds))
labels.sort()
columns = [f'Predicted {label}' for label in labels]
index = [f'Actual {label}' for label in labels]

# create confusion matrix
confusion = pd.DataFrame(confusion_matrix(y_test, y_preds),
                     columns=columns, index=index)
print(model_name + ' confusion matrix:')
print(confusion, '\n')

# append to confusion list
confusion_dict[model_name] = confusion